# Import Library

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Dense, Embedding, CuDNNLSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

# Import Text and Data Preprocessing

In [2]:
Scripts = ""
def TextToString(txt):
    with open(txt, "r") as file:
        data = file.readlines()
        script = ""
        for x in data[1:-1]:
            x = x.lower()
            x = x.replace('"', '')
            x = x.replace("\n", " \n ")
            x = x.split(' ')
            x[1] += ":"
            script += " ".join(x[1:-1]).replace("\n", " \n ")
        return script

Scripts += TextToString("datasets/SW_EpisodeIV.txt")
Scripts += TextToString("datasets/SW_EpisodeV.txt")
Scripts += TextToString("datasets/SW_EpisodeVI.txt")

In [3]:
print(Scripts[:1000])

threepio: did you hear that?  they've shut down the main reactor.  we'll be destroyed for sure.  this is madness!  
 threepio: we're doomed!  
 threepio: there'll be no escape for the princess this time.  
 threepio: what's that?  
 threepio: i should have known better than to trust the logic of a half-sized thermocapsulary dehousing assister...  
 luke: hurry up!  come with me!  what are you waiting for?!  get in gear!  
 threepio: artoo! artoo-detoo, where are you?  
 threepio: at last!  where have you been?  
 threepio: they're heading in this direction. what are we going to do?  we'll be sent to the spice mines of kessel or smashed into who knows what!  
 threepio: wait a minute, where are you going?  
 imperial: officer the death star plans are not in the main computer.  
 vader: where are those transmissions you intercepted?  
 rebel: officer we intercepted no transmissions. aaah...  this is a consular ship. were on a diplomatic mission.  
 vader: if this is a consular ship... wh

In [4]:
text_data = Scripts

charindex = list(set(text_data))
charindex.sort()

print(charindex)

['\n', ' ', '!', '#', "'", ',', '-', '.', '/', '0', '1', '2', '3', '4', '6', '7', '8', ':', ';', '?', '\\', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [6]:
char_size = len(charindex)
sequence_length = 100

X_train = []
Y_train = []

for i in range(0, len(text_data) - sequence_length, 1):
    X = text_data[i:i + sequence_length]
    Y = text_data[i + sequence_length]
    X_train.append([charindex.index(x) for x in X])
    Y_train.append(charindex.index(Y))

X_train = np.reshape(X_train, (len(X_train), sequence_length))
Y_train = to_categorical(Y_train)

In [ ]:
X_train

In [ ]:
Y_train.shape

# Model

In [ ]:
model = Sequential()
model.add(Input(shape=(sequence_length, )))
model.add(Embedding(char_size, 100))
model.add(CuDNNLSTM(512, return_sequences=True))
model.add(CuDNNLSTM(256, return_sequences=True))
model.add(CuDNNLSTM(128))
model.add(Dense(256, activation='elu'))
model.add(Dense(128, activation='elu'))
model.add(Dense(char_size, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['acc'])

In [ ]:
model.summary()

In [ ]:
history = model.fit(X_train, 
                    Y_train,
                    batch_size=64,
                    epochs=100,
                    verbose=1)

model.save_weights('weight.h5')

# Generating Data

In [ ]:
model.load_weights('weight.h5')

In [ ]:
from tqdm import tqdm
text_length = 5000
x = np.random.randint(0, len(X_train)-1)
pattern = X_train[x]

output = []
for t in tqdm(range(text_length)):
    x = np.reshape(pattern, (1, len(pattern)))
    prediction = model.predict(x)
    result = np.argmax(prediction)
    output.append(result)
    pattern = np.append(pattern, result)
    pattern = pattern[1:len(pattern)]
    

In [ ]:
output = [charindex[x] for x in output]
output = ''.join(output)

print(output)